### Imports

In [1]:
%run 'utils.ipynb'
%run 'SDNE+.ipynb'

In [2]:
import dgl
import tensorflow as tf
import torch

from tensorflow.keras import layers
from dgl.nn.tensorflow import GraphConv

from dgl.data import CiteseerGraphDataset


Metal device set to: Apple M1 Max


2022-11-09 17:30:07.134647: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-09 17:30:07.134765: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/zohairshafi/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data

In [3]:
with open('./email.pkl', 'rb') as file:
    graph_dict = pkl.load(file)
    
graph = nx.Graph(nx.to_numpy_array(graph_dict['graph']))
sense_feat_dict, sense_features = get_sense_features(graph, ppr_flag = 'std')

dgl_graph = dgl.from_networkx(graph)

986it [00:25, 38.44it/s]


/Users/zohairshafi/miniforge3/lib/python3.10/site-packages/networkx/algorithms/centrality/katz.py:325: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G, nodelist=nodelist, weight=weight).todense().T


### SDNE+

In [4]:
sdne = SDNE_plus(graph, 
                  hidden_size = [32, 64], 
                  lr = 1e-3,
                  sense_features = sense_features.astype(np.float32),
                  alpha = 0.1, 
                  beta = 10, 
                  gamma = 0, 
                  delta = 0)
history = sdne.train(epochs = 200, batch_size = 1024)
e = sdne.get_embeddings()
embed_og = np.array([e[node_name] for node_name in graph.nodes()])
embed_og = (embed_og - np.min(embed_og)) / np.ptp(embed_og)


feature_dict_og = find_feature_membership(input_embed = embed_og,
                                                    embed_name = 'SDNE',
                                                    sense_features = sense_features,
                                                    sense_feat_dict = sense_feat_dict,
                                                    top_k = 8,
                                                    solver = 'nmf')

explain_og = feature_dict_og['explain_norm']
explain_og = (explain_og - np.min(explain_og)) / np.ptp(explain_og)

1/1 [==============================] - ETA: 0s

2022-11-09 17:30:35.470942: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-09 17:30:35.499344: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 0s 14ms/step
batch_size(1024) > node_size(986),set batch_size = 986
Epoch 1/200


2022-11-09 17:30:35.724443: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-09 17:30:36.283560: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 976ms/step - loss: 1658.3174 - 2nd_loss: 1653.9438 - tf.clip_by_value_loss: 4.3417 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 36ms/step - loss: 1642.4193 - 2nd_loss: 1636.4971 - tf.clip_by_value_loss: 5.8904 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 3/200
1/1 [==============================] - 0s 31ms/step - loss: 1626.6416 - 2nd_loss: 1619.0836 - tf.clip_by_value_loss: 7.5260 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 4/200
1/1 [==============================] - 0s 37ms/step - loss: 1608.4221 - 2nd_loss: 1598.8717 - tf.clip_by_value_loss: 9.5185 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 5/200
1/1 [==============================] - 0s 42ms/step - loss: 1586.0382 - 2nd_loss: 1573.9587 - tf.clip_by_value_loss: 12.0477 - tf.math.reduce_sum_loss:

1/1 [==============================] - 0s 28ms/step - loss: 292.1122 - 2nd_loss: 271.6635 - tf.clip_by_value_loss: 20.4145 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 40/200
1/1 [==============================] - 0s 27ms/step - loss: 285.7651 - 2nd_loss: 267.8333 - tf.clip_by_value_loss: 17.8976 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 41/200
1/1 [==============================] - 0s 27ms/step - loss: 279.9421 - 2nd_loss: 260.7052 - tf.clip_by_value_loss: 19.2027 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 42/200
1/1 [==============================] - 0s 27ms/step - loss: 276.8729 - 2nd_loss: 261.0310 - tf.clip_by_value_loss: 15.8077 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 43/200
1/1 [==============================] - 0s 27ms/step - loss: 268.6497 - 2nd_loss: 250.7135 - tf.clip_by_value_loss: 17.9018 - tf.math.reduce_sum_loss: 0.

1/1 [==============================] - 0s 27ms/step - loss: 191.8138 - 2nd_loss: 183.9311 - tf.clip_by_value_loss: 7.8478 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 78/200
1/1 [==============================] - 0s 27ms/step - loss: 191.4115 - 2nd_loss: 184.1787 - tf.clip_by_value_loss: 7.1981 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 79/200
1/1 [==============================] - 0s 28ms/step - loss: 190.8791 - 2nd_loss: 183.1943 - tf.clip_by_value_loss: 7.6500 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 80/200
1/1 [==============================] - 0s 27ms/step - loss: 190.6029 - 2nd_loss: 183.5584 - tf.clip_by_value_loss: 7.0097 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 81/200
1/1 [==============================] - 0s 27ms/step - loss: 190.1611 - 2nd_loss: 182.6478 - tf.clip_by_value_loss: 7.4785 - tf.math.reduce_sum_loss: 0.0000e

1/1 [==============================] - 0s 27ms/step - loss: 180.3896 - 2nd_loss: 174.7855 - tf.clip_by_value_loss: 5.5693 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 116/200
1/1 [==============================] - 0s 27ms/step - loss: 180.5259 - 2nd_loss: 175.4651 - tf.clip_by_value_loss: 5.0261 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 117/200
1/1 [==============================] - 0s 27ms/step - loss: 180.2948 - 2nd_loss: 174.7557 - tf.clip_by_value_loss: 5.5042 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 118/200
1/1 [==============================] - 0s 28ms/step - loss: 180.4865 - 2nd_loss: 175.5087 - tf.clip_by_value_loss: 4.9431 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 119/200
1/1 [==============================] - 0s 28ms/step - loss: 180.2486 - 2nd_loss: 174.7731 - tf.clip_by_value_loss: 5.4407 - tf.math.reduce_sum_loss: 0.0

1/1 [==============================] - 0s 29ms/step - loss: 177.6519 - 2nd_loss: 173.1746 - tf.clip_by_value_loss: 4.4425 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 154/200
1/1 [==============================] - 0s 33ms/step - loss: 177.8942 - 2nd_loss: 173.8709 - tf.clip_by_value_loss: 3.9886 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 155/200
1/1 [==============================] - 0s 31ms/step - loss: 177.7481 - 2nd_loss: 173.3065 - tf.clip_by_value_loss: 4.4069 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 156/200
1/1 [==============================] - 0s 32ms/step - loss: 178.0517 - 2nd_loss: 174.0779 - tf.clip_by_value_loss: 3.9392 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 157/200
1/1 [==============================] - 0s 32ms/step - loss: 177.8849 - 2nd_loss: 173.4740 - tf.clip_by_value_loss: 4.3762 - tf.math.reduce_sum_loss: 0.0

1/1 [==============================] - 0s 27ms/step - loss: 175.9401 - 2nd_loss: 172.1920 - tf.clip_by_value_loss: 3.7135 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 192/200
1/1 [==============================] - 0s 28ms/step - loss: 176.0761 - 2nd_loss: 172.6606 - tf.clip_by_value_loss: 3.3809 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 193/200
1/1 [==============================] - 0s 26ms/step - loss: 175.9637 - 2nd_loss: 172.2427 - tf.clip_by_value_loss: 3.6863 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 194/200
1/1 [==============================] - 0s 27ms/step - loss: 176.1174 - 2nd_loss: 172.7345 - tf.clip_by_value_loss: 3.3483 - tf.math.reduce_sum_loss: 0.0000e+00 - tf.math.multiply_1_loss: 0.0000e+00
Epoch 195/200
1/1 [==============================] - 0s 27ms/step - loss: 176.0063 - 2nd_loss: 172.3108 - tf.clip_by_value_loss: 3.6609 - tf.math.reduce_sum_loss: 0.0

In [5]:
sdne_plus_init = SDNE_plus(graph, 
                          hidden_size = [32, 64], 
                          lr = 9e-4,
                          sense_features = sense_features.astype(np.float32),
                          alpha = 10, 
                          beta = 1, 
                          gamma = 100, 
                          delta = 100)

sdne_plus_init.model.set_weights(sdne.model.get_weights())
history = sdne_plus_init.train(epochs = 400, batch_size = 1024)
e = sdne_plus_init.get_embeddings()
embed_plus_init = np.array([e[node_name] for node_name in graph.nodes()])
embed_plus_init = (embed_plus_init - np.min(embed_plus_init)) / np.ptp(embed_plus_init)


feature_dict_plus_init = find_feature_membership(input_embed = embed_plus_init,
                                                    embed_name = 'SDNE+ Init',
                                                    sense_features = sense_features,
                                                    sense_feat_dict = sense_feat_dict,
                                                    top_k = 8,
                                                    solver = 'nmf')

explain_plus_init = feature_dict_plus_init['explain_norm']
explain_plus_init = (explain_plus_init - np.min(explain_plus_init)) / np.ptp(explain_plus_init)

1/1 [==============================] - 0s 9ms/step


2022-11-09 17:30:44.584983: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 0s 10ms/step
batch_size(1024) > node_size(986),set batch_size = 986
Epoch 1/400


2022-11-09 17:30:44.881320: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-09 17:30:45.354141: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 892ms/step - loss: 3233.6519 - 2nd_loss: 1.7265 - tf.clip_by_value_1_loss: 359.4109 - tf.math.reduce_sum_1_loss: 1835.2456 - tf.math.multiply_3_loss: 1037.2343
Epoch 2/400
1/1 [==============================] - 0s 44ms/step - loss: 3090.4475 - 2nd_loss: 1.7210 - tf.clip_by_value_1_loss: 358.8837 - tf.math.reduce_sum_1_loss: 1725.7926 - tf.math.multiply_3_loss: 1004.0156
Epoch 3/400
1/1 [==============================] - 0s 40ms/step - loss: 2999.2876 - 2nd_loss: 1.7355 - tf.clip_by_value_1_loss: 365.9018 - tf.math.reduce_sum_1_loss: 1651.1659 - tf.math.multiply_3_loss: 980.4500
Epoch 4/400
1/1 [==============================] - 0s 37ms/step - loss: 2921.4336 - 2nd_loss: 1.7499 - tf.clip_by_value_1_loss: 364.9919 - tf.math.reduce_sum_1_loss: 1592.0527 - tf.math.multiply_3_loss: 962.6044
Epoch 5/400
1/1 [==============================] - 0s 46ms/step - loss: 2849.3752 - 2nd_loss: 1.7584 - tf.clip_by_value_1_loss: 373.4041 - tf.math.reduce_sum_1_l

...

1/1 [==============================] - 0s 66ms/step - loss: 388.6137 - 2nd_loss: 1.7009 - tf.clip_by_value_1_loss: 57.0681 - tf.math.reduce_sum_1_loss: 88.5300 - tf.math.multiply_3_loss: 241.2764
Epoch 269/400
1/1 [==============================] - 0s 28ms/step - loss: 387.4741 - 2nd_loss: 1.7012 - tf.clip_by_value_1_loss: 58.8872 - tf.math.reduce_sum_1_loss: 87.3841 - tf.math.multiply_3_loss: 239.4633
Epoch 270/400
1/1 [==============================] - 0s 39ms/step - loss: 386.3942 - 2nd_loss: 1.7011 - tf.clip_by_value_1_loss: 56.7820 - tf.math.reduce_sum_1_loss: 87.7065 - tf.math.multiply_3_loss: 240.1663
Epoch 271/400
1/1 [==============================] - 0s 27ms/step - loss: 385.3744 - 2nd_loss: 1.7014 - tf.clip_by_value_1_loss: 58.6667 - tf.math.reduce_sum_1_loss: 86.5864 - tf.math.multiply_3_loss: 238.3816
Epoch 272/400
1/1 [==============================] - 0s 27ms/step - loss: 384.4259 - 2nd_loss: 1.7016 - tf.clip_by_value_1_loss: 56.6987 - tf.math.reduce_sum_1_loss: 86.9042 

1/1 [==============================] - 0s 31ms/step - loss: 350.5145 - 2nd_loss: 1.6970 - tf.clip_by_value_1_loss: 52.1251 - tf.math.reduce_sum_1_loss: 74.8154 - tf.math.multiply_3_loss: 221.8385
Epoch 308/400
1/1 [==============================] - 0s 29ms/step - loss: 349.3889 - 2nd_loss: 1.6962 - tf.clip_by_value_1_loss: 51.7565 - tf.math.reduce_sum_1_loss: 74.4449 - tf.math.multiply_3_loss: 221.4528
Epoch 309/400
1/1 [==============================] - 0s 34ms/step - loss: 348.3118 - 2nd_loss: 1.6961 - tf.clip_by_value_1_loss: 51.4073 - tf.math.reduce_sum_1_loss: 74.2112 - tf.math.multiply_3_loss: 220.9587
Epoch 310/400
1/1 [==============================] - 0s 27ms/step - loss: 347.3103 - 2nd_loss: 1.6955 - tf.clip_by_value_1_loss: 51.2194 - tf.math.reduce_sum_1_loss: 73.8251 - tf.math.multiply_3_loss: 220.5318
Epoch 311/400
1/1 [==============================] - 0s 29ms/step - loss: 346.3914 - 2nd_loss: 1.6957 - tf.clip_by_value_1_loss: 50.9618 - tf.math.reduce_sum_1_loss: 73.6134 

1/1 [==============================] - 0s 30ms/step - loss: 327.2771 - 2nd_loss: 1.7093 - tf.clip_by_value_1_loss: 53.9704 - tf.math.reduce_sum_1_loss: 64.7597 - tf.math.multiply_3_loss: 206.7991
Epoch 347/400
1/1 [==============================] - 0s 28ms/step - loss: 324.7918 - 2nd_loss: 1.7031 - tf.clip_by_value_1_loss: 53.4763 - tf.math.reduce_sum_1_loss: 64.1247 - tf.math.multiply_3_loss: 205.4492
Epoch 348/400
1/1 [==============================] - 0s 28ms/step - loss: 322.6479 - 2nd_loss: 1.7010 - tf.clip_by_value_1_loss: 50.6839 - tf.math.reduce_sum_1_loss: 64.2529 - tf.math.multiply_3_loss: 205.9715
Epoch 349/400
1/1 [==============================] - 0s 28ms/step - loss: 320.6734 - 2nd_loss: 1.6972 - tf.clip_by_value_1_loss: 50.4415 - tf.math.reduce_sum_1_loss: 63.6995 - tf.math.multiply_3_loss: 204.7966
Epoch 350/400
1/1 [==============================] - 0s 27ms/step - loss: 318.9633 - 2nd_loss: 1.6952 - tf.clip_by_value_1_loss: 48.3600 - tf.math.reduce_sum_1_loss: 63.7395 

1/1 [==============================] - 0s 28ms/step - loss: 321.0325 - 2nd_loss: 1.7441 - tf.clip_by_value_1_loss: 71.4672 - tf.math.reduce_sum_1_loss: 55.9218 - tf.math.multiply_3_loss: 191.8608
Epoch 386/400
1/1 [==============================] - 0s 29ms/step - loss: 324.1638 - 2nd_loss: 1.7606 - tf.clip_by_value_1_loss: 72.5083 - tf.math.reduce_sum_1_loss: 56.4943 - tf.math.multiply_3_loss: 193.3620
Epoch 387/400
1/1 [==============================] - 0s 30ms/step - loss: 324.2520 - 2nd_loss: 1.7534 - tf.clip_by_value_1_loss: 75.6198 - tf.math.reduce_sum_1_loss: 55.5801 - tf.math.multiply_3_loss: 191.2599
Epoch 388/400
1/1 [==============================] - 0s 29ms/step - loss: 324.1287 - 2nd_loss: 1.7669 - tf.clip_by_value_1_loss: 73.4019 - tf.math.reduce_sum_1_loss: 56.1467 - tf.math.multiply_3_loss: 192.7745
Epoch 389/400
1/1 [==============================] - 0s 28ms/step - loss: 322.4652 - 2nd_loss: 1.7523 - tf.clip_by_value_1_loss: 74.7481 - tf.math.reduce_sum_1_loss: 55.2422 

/var/folders/g7/2jc8pp3979bb81nstc9qs3900000gn/T/ipykernel_60457/739605159.py:108: RuntimeWarning: overflow encountered in exp
  explain_norm_softmax = np.array([np.exp(x) / sum(np.exp(x)) for x in explain_norm])
/var/folders/g7/2jc8pp3979bb81nstc9qs3900000gn/T/ipykernel_60457/739605159.py:108: RuntimeWarning: invalid value encountered in divide
  explain_norm_softmax = np.array([np.exp(x) / sum(np.exp(x)) for x in explain_norm])


In [6]:
Y = embed_plus_init
sense_mat = tf.einsum('ij, ik -> ijk', Y, sense_features)
y_norm = tf.linalg.diag_part(tf.matmul(Y, Y, transpose_b = True), k = 0)
sense_norm = tf.linalg.diag_part(tf.matmul(sense_features, sense_features, transpose_b = True), k = 0)
norm = y_norm * tf.cast(sense_norm, tf.float32)
D = tf.transpose(tf.transpose(sense_mat) / norm)

In [7]:
feature_shape = D.shape[1] * D.shape[2]
feature_matrix = np.zeros((D.shape[0], feature_shape))

for idx in tqdm(range(D.shape[0])):
    feature_matrix[idx, :] = np.ravel(D[idx])

100%|████████████████████████████████████████| 986/986 [00:04<00:00, 198.36it/s]


### Model

In [8]:
class GCN(tf.keras.Model):
    def __init__(self, g, in_feats, n_hidden, n_classes, n_layers, activation, dropout):
        
        super(GCN, self).__init__()
        self.g = g
        self.layer_list = []
        
        # Input layer
        self.layer_list.append(GraphConv(in_feats, n_hidden, activation = activation))
        
        # Hidden layers
        for i in range(n_layers - 1):
            self.layer_list.append(GraphConv(n_hidden, n_hidden, activation = activation))
            
        # Output layer
        self.layer_list.append(GraphConv(n_hidden, n_classes))
        self.dropout = layers.Dropout(dropout)

    def call(self, features):
        h = features
        for i, layer in enumerate(self.layer_list):
            if i != 0:
                h = self.dropout(h)
            h = layer(self.g, h)
        return h


In [13]:
feature_matrix = torch.from_numpy(feature_matrix)

In [17]:
feature_matrix.to('cpu:0')

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [18]:
dgl_graph.ndata['feat'] = feature_matrix

AttributeError: 'torch.device' object has no attribute 'split'

In [45]:
model = GCN(g = dgl_graph,
            in_feats = feature_shape, # d x f
            n_hidden = 512,
            n_classes = feature_shape, # d x f
            n_layers = 2,
            activation = tf.nn.relu,
            dropout = 0.3,
        )

In [46]:
model(feature_matrix)

DGLError: Exception encountered when calling layer "graph_conv_12" "                 f"(type GraphConv).

Cannot assign node feature "h" on device /gpu:0 to a graph on device /cpu:0. Call DGLGraph.to() to copy the graph to the same device.

Call arguments received by layer "graph_conv_12" "                 f"(type GraphConv):
  • graph=Graph(num_nodes=986, num_edges=32751,
      ndata_schemes={}
      edata_schemes={})
  • feat=tensor([[1.6951e-02, 3.7704e-02, 1.4208e-02,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.0233e-02, 3.6417e-02, 1.7506e-02,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.8689e-02, 2.0487e-02, 1.4017e-02,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 5.7889e-05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 6.0959e-05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 2.5287e-05,  ..., 0.0000e+00, 0.0000e